In [2]:
import torch
import pandas as pd
from sentence_transformers import SentenceTransformer

print("Torch OK:", torch.__version__)
print("Pandas OK:", pd.__version__)

model = SentenceTransformer("all-MiniLM-L6-v2")
print("SBERT OK ✅")

Torch OK: 2.10.0+cpu
Pandas OK: 2.3.3


Loading weights: 100%|█████████████████████| 103/103 [00:00<00:00, 393.94it/s, Materializing param=pooler.dense.weight]
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


SBERT OK ✅


In [3]:
import pandas as pd
import numpy as np
import re
import os
from sentence_transformers import SentenceTransformer

In [5]:
jobs = pd.read_csv(r"C:\Users\jhanvi.kasundra\Downloads\Projects New\Data/jobs_clean.csv")
resumes = pd.read_csv(r"C:\Users\jhanvi.kasundra\Downloads\Projects New\Data/resumes_clean.csv")

In [8]:
jobs = jobs.reset_index(drop=True)
jobs["job_id"] = jobs.index

resumes = resumes.reset_index(drop=True)
resumes["resumes_id"] = resumes.index

print("jobs:",jobs.shape, "| resumes:", resumes.shape)
print("columns resumes:", resumes.columns.tolist())

jobs: (15405, 6) | resumes: (13389, 3)
columns resumes: ['category', 'text_cv', 'resumes_id']


In [10]:
resumes.head()

,category,text_cv,resumes_id
0,accountant,education\r\nomba - executive leadership unive...,0
1,accountant,"howard gerrard\r\naccountant\r\ndeyjob.com, bi...",1
2,accountant,kevin frank\r\nsenior accountant\r\ninfo@resum...,2
3,accountant,place of birth\r\nnationality\r\nolivia ogilvy...,3
4,accountant,"stephen greet, cpa\r\nsenior accountant\r\nthr...",4


In [11]:
print(resumes["category"].value_counts().head(20))

category
education                    410
electrical engineering       384
mechanical engineer          384
consultant                   368
sales                        364
civil engineer               364
management                   361
human resources              360
digital media                358
accountant                   350
java developer               348
operations manager           345
building and construction    345
testing                      344
architecture                 344
aviation                     340
business analyst             340
finance                      339
sql developer                338
public relations             337
Name: count, dtype: int64


In [12]:
import re

EXACT_MAP = {
    "human resources": "hr",
    "hr": "hr",
    "accountant": "finance",
    "finance": "finance",
    "banking": "banking",
    "digital media": "digital media",
    "public relations": "marketing",
    "business analyst": "business analyst",
    "pmo": "business analyst",
    "consultant": "consulting",
    "management": "management",
    "operations manager": "management",
    "information technology": "information technology",
    "it": "information technology",
    "data science": "data science",
    "database": "information technology",
    "devops": "information technology",
    "etl developer": "information technology",
    "sql developer": "information technology",
    "python developer": "information technology",
    "dotnet developer": "information technology",
    "java developer": "information technology",
    "react developer": "information technology",
    "sap developer": "information technology",
    "testing": "information technology",
    "network security engineer": "information technology",
    "blockchain": "information technology",
    "building and construction": "engineering",
    "civil engineer": "engineering",
    "mechanical engineer": "engineering",
    "electrical engineering": "engineering",
    "architecture": "engineering",
    "aviation": "engineering",
    "automobile": "engineering",
    "arts": "design",
    "designing": "design",
    "web designing": "design",
    "apparel": "retail",
    "food and beverages": "hospitality",
    "bpo": "operations",
    "advocate": "legal",
}

RULES = [
    (r"(data|ml|machine learning|ai|analytics|business intelligence|bi\b|database|sql|etl|devops|cloud|aws|azure|gcp|python|java|dotnet|react|sap|testing|qa|security|network)", "information technology"),
    (r"(finance|financial|account|accountant|audit|tax|treasury|fp&a|risk|credit|bank|banking|insurance)", "finance"),
    (r"(marketing|digital|seo|sem|content|social media|brand|advertising|media|public relations|communications|crm)", "marketing"),
    (r"(sales|business development|bd|account executive|inside sales|customer success|client success)", "sales"),
    (r"(human resources|hr\b|recruit|talent|workforce|people)", "hr"),
    (r"(health|healthcare|clinical|hospital|pharma|medical|patient|claims|fitness|wellness|nutrition)", "healthcare"),
    (r"(education|teacher|teaching|faculty|professor|lecturer|school|university|college|trainer)", "education"),
    (r"(legal|law|advocate|attorney|paralegal|litigation|contract)", "legal"),
    (r"(civil|mechanical|electrical|electronics|aviation|aerospace|automobile|automotive|architecture|construction|building)", "engineering"),
    (r"(design|designer|ui|ux|graphic|web designing|arts|creative)", "design"),
    (r"(agriculture|agri|farming|crop|environment|sustainability)", "agriculture"),
    (r"(bpo|call center|customer service|support|operations|back office)", "operations"),
    (r"(food|beverage|restaurant|hospitality|chef|cooking)", "hospitality"),
    (r"(apparel|fashion|retail|merchandising)", "retail"),
]

def normalize_category(cat: str) -> str:
    cat = str(cat).strip().lower()
    if cat in EXACT_MAP:
        return EXACT_MAP[cat]
    for pattern, label in RULES:
        if re.search(pattern, cat):
            return label
    return "other"

resumes["category_norm"] = resumes["category"].apply(normalize_category)

print(resumes["category_norm"].value_counts().head(20))
print("other count:", resumes["category_norm"].value_counts().get("other", 0))


category_norm
information technology    3584
engineering               2474
design                     899
management                 706
finance                    689
business analyst           626
education                  410
consulting                 368
sales                      364
hr                         360
digital media              358
marketing                  337
healthcare                 332
retail                     320
banking                    314
data science               299
agriculture                293
legal                      291
operations                 203
hospitality                162
Name: count, dtype: int64
other count: 0


In [13]:
CATEGORY_FILTERS = {
    "information technology": r"(?:software|cloud|aws|azure|gcp|devops|data engineer|etl|pipeline|kafka|spark|api|ml|llm|nlp|ci/cd|docker|kubernetes)",
    "finance": r"(?:finance|financial|accounting|audit|tax|fp&a|investment|portfolio|credit|risk|treasury|compliance)",
    "banking": r"(?:bank|banking|loan|mortgage|credit|risk|fraud|treasury)",
    "digital media": r"(?:marketing|seo|sem|paid media|campaign|ads|advertising|social media|content|brand|ecommerce)",
    "marketing": r"(?:marketing|brand|content|advertising|campaign|crm|email marketing|growth)",
    "hr": r"(?:recruiting|talent|hr analytics|compensation|benefits|workforce|people analytics)",
    "business analyst": r"(?:requirements|stakeholder|dashboards|reporting|analytics|product|kpi|agile|scrum)",
    "management": r"(?:operations|strategy|planning|forecast|budget|program management|project management|leadership)",
    "engineering": r"(?:engineering|civil|mechanical|electrical|construction|architecture|automotive|aviation|aerospace|manufacturing)",
    "design": r"(?:design|designer|ui|ux|graphic|creative|figma|adobe)",
    "sales": r"(?:sales|business development|account executive|pipeline|crm|customer success)",
    "consulting": r"(?:consultant|consulting|advisory|client|engagement|strategy)",
    "healthcare": r"(?:health|clinical|hospital|pharma|medical|patient|claims|ehr|emr|hipaa)",
    "legal": r"(?:legal|law|advocate|attorney|paralegal|contract|litigation)",
    "operations": r"(?:operations|support|call center|customer service|process|workflow)",
    "education": r"(?:education|teacher|school|trainer|curriculum|student)",
    "agriculture": r"(?:agriculture|farming|crop|soil|yield|agri)",
    "retail": r"(?:retail|merchandising|fashion|apparel|store|inventory|ecommerce)",
    "hospitality": r"(?:hospitality|restaurant|food|beverage|hotel)",
    "data science": r"(?:data scientist|machine learning|ml|modeling|statistics|nlp|deep learning|forecasting|prediction)",
}

In [15]:
model = SentenceTransformer("all-MiniLM-L6-v2")

Loading weights: 100%|█████████████████████| 103/103 [00:00<00:00, 300.98it/s, Materializing param=pooler.dense.weight]
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [16]:
test = model.encode(["hello world", "machine learning"])
print(test.shape)

(2, 384)


In [17]:
import numpy as np

def cosine_sim_matrix(A, B):
    A = A / np.linalg.norm(A, axis=1, keepdims=True)
    B = B / np.linalg.norm(B, axis=1, keepdims=True)
    return A @ B.T

In [18]:
import os

JOB_EMB_PATH = f"{DATA_PATH}/job_embeddings_allMiniLM.npy"

if os.path.exists(JOB_EMB_PATH):
    job_emb_all = np.load(JOB_EMB_PATH)
    print("Loaded cached job embeddings:", job_emb_all.shape)
else:
    print("Computing job embeddings (one-time)...")
    job_emb_all = model.encode(
        jobs["job_text"].tolist(),
        batch_size=64,
        show_progress_bar=True
    )
    job_emb_all = np.array(job_emb_all)
    np.save(JOB_EMB_PATH, job_emb_all)
    print("Saved cached job embeddings:", job_emb_all.shape)

Computing job embeddings (one-time)...


Batches: 100%|███████████████████████████████████████████████████████████████████████| 241/241 [07:53<00:00,  1.97s/it]

Saved cached job embeddings: (15405, 384)


In [20]:
results = []
top_k = 5
MAX_JOBS_PER_CATEGORY = 5000

for cat, group in resumes.groupby("category_norm"):
    print("\n===========================")
    print(f"Category: {cat} | resumes: {len(group)}")

    jobs_cat = jobs.copy()

    # Narrow job pool for this category
    if cat in CATEGORY_FILTERS:
        pattern = CATEGORY_FILTERS[cat]
        mask = jobs_cat["job_text"].str.lower().str.contains(pattern, regex=True, na=False)
        jobs_cat = jobs_cat[mask].copy()
        print("Jobs after filter:", len(jobs_cat))
    else:
        print("No category filter -> global pool")

    # fallback if too few
    if len(jobs_cat) < 300:
        jobs_cat = jobs.copy()
        print("Fallback to global pool:", len(jobs_cat))

    # cap for speed
    if MAX_JOBS_PER_CATEGORY and len(jobs_cat) > MAX_JOBS_PER_CATEGORY:
        jobs_cat = jobs_cat.sample(MAX_JOBS_PER_CATEGORY, random_state=42).copy()
        print("Capped jobs to:", len(jobs_cat))

    group = group.reset_index(drop=True)

    # Pull cached job embeddings by job_id
    job_ids = jobs_cat["job_id"].values
    job_emb = job_emb_all[job_ids]

    # Compute resume embeddings for this category chunk
    res_emb = model.encode(
        group["text_cv"].tolist(),
        batch_size=64,
        show_progress_bar=True
    )
    res_emb = np.array(res_emb)

    sim = cosine_sim_matrix(res_emb, job_emb)

    # Save top matches
    for i in range(len(group)):
        best = sim[i].argsort()[::-1][:top_k]
        for rank, j in enumerate(best, start=1):
            jj = int(job_ids[j])
            results.append({
                "resumes_id": int(group.loc[i, "resumes_id"]),
                "resume_category_norm": cat,
                "rank": rank,
                "score": float(sim[i, j]),
                "job_id": jj,
                "job_title": jobs.loc[jj, "job_title"],
                "company_name": jobs.loc[jj].get("company_name", None),
                "location": jobs.loc[jj].get("location", None),
                "source": jobs.loc[jj, "source"],
            })

out_sbert = pd.DataFrame(results).sort_values(["resumes_id", "rank"])
print("SBERT matches shape:", out_sbert.shape)
out_sbert.head(10)



Category: agriculture | resumes: 293
Jobs after filter: 260
Fallback to global pool: 15405
Capped jobs to: 5000


Batches: 100%|███████████████████████████████████████████████████████████████████████████| 5/5 [00:08<00:00,  1.73s/it]



Category: banking | resumes: 314
Jobs after filter: 4019


Batches: 100%|███████████████████████████████████████████████████████████████████████████| 5/5 [00:09<00:00,  1.83s/it]



Category: business analyst | resumes: 626
Jobs after filter: 13611
Capped jobs to: 5000


...

Batches: 100%|███████████████████████████████████████████████████████████████████████████| 5/5 [00:09<00:00,  2.00s/it]



Category: sales | resumes: 364
Jobs after filter: 4476


Batches: 100%|███████████████████████████████████████████████████████████████████████████| 6/6 [00:11<00:00,  1.95s/it]


SBERT matches shape: (66945, 9)


,resumes_id,resume_category_norm,rank,score,job_id,job_title,company_name,location,source
30205,0,finance,1,0.615372,6898,Senior Accountant - High Growth Real Estate Firm,NaN,NaN,df1
30206,0,finance,2,0.615372,6521,Senior Accountant - High Growth Real Estate Firm,NaN,NaN,df1
30207,0,finance,3,0.598011,8150,Corporate Accountant I or II - Financial Accou...,NaN,NaN,df1
30208,0,finance,4,0.581754,7738,Functional Analyst (CECL Data),NaN,NaN,df1
30209,0,finance,5,0.574828,13022,Mortgage Accounting Analyst,NaN,NaN,df1
30210,1,finance,1,0.700060,14256,Accounts Receivable Clerk / Accounting Analyst,NaN,NaN,df1
30211,1,finance,2,0.694162,13358,Accountant's Needed - Real Estate/Proper...,NaN,NaN,df1
30212,1,finance,3,0.670482,12004,Junior Data Analyst,NaN,NaN,df1
30213,1,finance,4,0.667984,9139,Senior Accounting Analyst,NaN,NaN,df1
30214,1,finance,5,0.665960,14067,Statutory Reporting Manager,NaN,NaN,df1


In [21]:
SBERT_MATCH_PATH = r"C:\Users\jhanvi.kasundra\Downloads\Projects New\Data/matches_sbert_all.csv"
out_sbert.to_csv(SBERT_MATCH_PATH, index=False)
print("Saved:", SBERT_MATCH_PATH)

Saved: C:\Users\jhanvi.kasundra\Downloads\Projects New\Data/matches_sbert_all.csv


In [22]:
print(out_sbert["score"].describe())
print(out_sbert.groupby("resume_category_norm")["score"].mean().sort_values(ascending=False).head(10))

count    66945.000000
mean         0.590971
std          0.079150
min          0.222909
25%          0.538375
50%          0.593018
75%          0.645859
max          0.866061
Name: score, dtype: float64
resume_category_norm
business analyst          0.657337
finance                   0.639255
information technology    0.638656
banking                   0.613090
consulting                0.612598
hr                        0.606603
digital media             0.598932
data science              0.597956
management                0.594402
sales                     0.584654
Name: score, dtype: float64


### Skill Gap for ALL categories (category-aware)

In [23]:
out_sbert = pd.read_csv(r"C:\Users\jhanvi.kasundra\Downloads\Projects New\Data/matches_sbert_all.csv")
print(out_sbert.shape)
out_sbert.head(3)

(66945, 9)


,resumes_id,resume_category_norm,rank,score,job_id,job_title,company_name,location,source
0,0,finance,1,0.615372,6898,Senior Accountant - High Growth Real Estate Firm,NaN,NaN,df1
1,0,finance,2,0.615372,6521,Senior Accountant - High Growth Real Estate Firm,NaN,NaN,df1
2,0,finance,3,0.598011,8150,Corporate Accountant I or II - Financial Accou...,NaN,NaN,df1


### define categories aware skill libraries

In [24]:
SKILLS_BY_CATEGORY = {
    "information technology": [
        "python","sql","excel","tableau","power bi","r",
        "aws","azure","gcp","snowflake","databricks","spark","kafka","airflow","dbt",
        "etl","data pipeline","data modeling","dimensional modeling",
        "pandas","numpy","scikit-learn","tensorflow","pytorch",
        "nlp","bert","llm","prompt engineering",
        "git","ci/cd","docker","kubernetes","api","microservices"
    ],
    "data science": [
        "python","sql","statistics","regression","classification","clustering",
        "time series","forecasting","feature engineering",
        "model evaluation","xgboost","lightgbm",
        "scikit-learn","tensorflow","pytorch",
        "nlp","llm","bert","prompt engineering",
        "mlops","docker","git"
    ],
    "finance": [
        "excel","sql","python","power bi","tableau",
        "financial modeling","fp&a","forecasting","budgeting",
        "variance analysis","valuation","risk management",
        "credit risk","market risk","liquidity risk",
        "gaap","ifrs","audit","treasury","compliance"
    ],
    "banking": [
        "excel","sql","python","power bi","tableau",
        "credit risk","underwriting","loan","mortgage","fraud",
        "kyc","aml","compliance","model risk",
        "portfolio","treasury","risk management"
    ],
    "business analyst": [
        "requirements gathering","stakeholder management","user stories",
        "agile","scrum","jira",
        "sql","excel","power bi","tableau",
        "kpi","dashboards","process mapping","gap analysis"
    ],
    "management": [
        "strategy","roadmap","program management","project management",
        "budgeting","forecasting","kpi","stakeholder management",
        "agile","scrum"
    ],
    "hr": [
        "recruiting","talent acquisition","workforce planning",
        "compensation","benefits","hris",
        "people analytics","employee engagement",
        "excel","power bi","tableau","sql"
    ],
    "digital media": [
        "google analytics","ga4","seo","sem","paid media",
        "campaign management","a/b testing",
        "meta ads","google ads","tag manager",
        "content","social media","email marketing","crm",
        "excel","sql"
    ],
    "marketing": [
        "google analytics","ga4","seo","sem","paid media",
        "campaign management","marketing analytics","attribution",
        "a/b testing","email marketing","crm","content","brand",
        "excel","sql","tableau","power bi"
    ],
    "sales": [
        "crm","salesforce","pipeline","forecasting","lead generation",
        "account management","negotiation","customer success",
        "excel","presentation","stakeholder management"
    ],
    "consulting": [
        "client management","stakeholder management","strategy",
        "problem solving","presentation","storytelling",
        "sql","excel","power bi","tableau"
    ],
    "healthcare": [
        "hipaa","claims","ehr","emr","icd",
        "clinical","patient","payer","provider",
        "excel","sql","power bi","tableau"
    ],
    "legal": [
        "contract","compliance","litigation","legal research",
        "documentation","negotiation"
    ],
    "operations": [
        "process improvement","kpi","dashboards",
        "excel","sql","power bi","tableau",
        "lean","six sigma","root cause analysis"
    ],
    "retail": [
        "merchandising","inventory","pricing","demand planning",
        "excel","sql","power bi","tableau","ecommerce"
    ],
    "hospitality": [
        "customer service","operations","scheduling",
        "inventory","budgeting"
    ],
    "education": [
        "curriculum","training","teaching","assessment",
        "learning management system","lms"
    ],
    "agriculture": [
        "crop","yield","soil","sustainability","forecasting"
    ],
}

# Fallback skills if category not found (should rarely happen since other=0)
FALLBACK_SKILLS = [
    "excel","sql","python","tableau","power bi",
    "stakeholder management","communication","presentation"
]

def extract_skills(text, skill_list):
    text = str(text).lower()
    found = set()
    for s in skill_list:
        # robust boundary matching
        pattern = r"(?<!\w)" + re.escape(s.lower()) + r"(?!\w)"
        if re.search(pattern, text):
            found.add(s)
    return found


### build skill gap rows 

In [27]:
rows = []

# helpful lookups (fast)
jobs_lookup = jobs.set_index("job_id")
res_lookup = resumes.set_index("resumes_id")

for _, m in out_sbert.iterrows():
    rid = int(m["resumes_id"])
    jid = int(m["job_id"])
    cat = m["resume_category_norm"]

    resume_text = res_lookup.loc[rid, "text_cv"]
    job_text = jobs_lookup.loc[jid, "job_text"]

    skill_list = SKILLS_BY_CATEGORY.get(cat, FALLBACK_SKILLS)

    r_sk = extract_skills(resume_text, skill_list)
    j_sk = extract_skills(job_text, skill_list)

    rows.append({
        **m.to_dict(),
        "skills_used_for_gap": "; ".join(skill_list),
        "resume_skills": ", ".join(sorted(r_sk)),
        "job_skills": ", ".join(sorted(j_sk)),
        "missing_skills": ", ".join(sorted(j_sk - r_sk)),
        "missing_skills_count": len(j_sk - r_sk)
    })

matches_sbert_gap = pd.DataFrame(rows)
matches_sbert_gap.head(10)


,resumes_id,resume_category_norm,rank,score,job_id,job_title,company_name,location,source,skills_used_for_gap,resume_skills,job_skills,missing_skills,missing_skills_count
0,0,finance,1,0.615372,6898,Senior Accountant - High Growth Real Estate Firm,NaN,NaN,df1,excel; sql; python; power bi; tableau; financi...,"compliance, financial modeling, sql, valuation","excel, gaap, valuation","excel, gaap",2
1,0,finance,2,0.615372,6521,Senior Accountant - High Growth Real Estate Firm,NaN,NaN,df1,excel; sql; python; power bi; tableau; financi...,"compliance, financial modeling, sql, valuation","excel, gaap, valuation","excel, gaap",2
2,0,finance,3,0.598011,8150,Corporate Accountant I or II - Financial Accou...,NaN,NaN,df1,excel; sql; python; power bi; tableau; financi...,"compliance, financial modeling, sql, valuation","budgeting, forecasting","budgeting, forecasting",2
3,0,finance,4,0.581754,7738,Functional Analyst (CECL Data),NaN,NaN,df1,excel; sql; python; power bi; tableau; financi...,"compliance, financial modeling, sql, valuation","credit risk, sql",credit risk,1
4,0,finance,5,0.574828,13022,Mortgage Accounting Analyst,NaN,NaN,df1,excel; sql; python; power bi; tableau; financi...,"compliance, financial modeling, sql, valuation",excel,excel,1
5,1,finance,1,0.700060,14256,Accounts Receivable Clerk / Accounting Analyst,NaN,NaN,df1,excel; sql; python; power bi; tableau; financi...,,excel,excel,1
6,1,finance,2,0.694162,13358,Accountant's Needed - Real Estate/Proper...,NaN,NaN,df1,excel; sql; python; power bi; tableau; financi...,,audit,audit,1
7,1,finance,3,0.670482,12004,Junior Data Analyst,NaN,NaN,df1,excel; sql; python; power bi; tableau; financi...,,"excel, financial modeling, power bi, python, s...","excel, financial modeling, power bi, python, s...",6
8,1,finance,4,0.667984,9139,Senior Accounting Analyst,NaN,NaN,df1,excel; sql; python; power bi; tableau; financi...,,"audit, compliance, excel, gaap, treasury","audit, compliance, excel, gaap, treasury",5
9,1,finance,5,0.665960,14067,Statutory Reporting Manager,NaN,NaN,df1,excel; sql; python; power bi; tableau; financi...,,gaap,gaap,1


In [28]:
OUT_PATH = r"C:\Users\jhanvi.kasundra\Downloads\Projects New\Data/matches_sbert_with_skill_gap.csv"
matches_sbert_gap.to_csv(OUT_PATH, index=False)
print("Saved:", OUT_PATH)
print(matches_sbert_gap["missing_skills_count"].describe())


Saved: C:\Users\jhanvi.kasundra\Downloads\Projects New\Data/matches_sbert_with_skill_gap.csv
count    66945.000000
mean         1.242005
std          1.504546
min          0.000000
25%          0.000000
50%          1.000000
75%          2.000000
max         13.000000
Name: missing_skills_count, dtype: float64


In [29]:
print(
    matches_sbert_gap.groupby("resume_category_norm")["missing_skills_count"]
    .mean()
    .sort_values(ascending=False)
    .head(15)
)

# show examples from marketing / finance / IT
for c in ["digital media", "marketing", "finance", "information technology"]:
    print("\n---", c, "---")
    print(matches_sbert_gap[matches_sbert_gap["resume_category_norm"]==c][
        ["score","job_title","missing_skills"]
    ].head(5))


resume_category_norm
information technology    1.835324
data science              1.684281
hr                        1.650556
marketing                 1.561424
banking                   1.456051
finance                   1.395356
healthcare                1.306024
digital media             1.268156
business analyst          1.136422
retail                    1.114375
design                    1.072303
consulting                1.059239
engineering               0.935893
sales                     0.842857
operations                0.681773
Name: missing_skills_count, dtype: float64

--- digital media ---
         score                                     job_title missing_skills
5620  0.679388                     Digital Marketing Analyst            ga4
5621  0.679388                     Digital Marketing Analyst            ga4
5622  0.624903  Marketing Specialist - Paid Search - SEM/SEO               
5623  0.624903  Marketing Specialist - Paid Search - SEM/SEO               
5624  0.